In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os 
import cv2
from random import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras import activations
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
GOOD_POSE_DIR = 'good_pose'
BAD_POSE_DIR = 'bad_pose'
IMG_SIZE = 256

In [ ]:
def label_image(image):
    """Function to encode and give a label to an image.
    
    Arguments:
        image {string} -- Represents the image name.

    Returns
        integer -- 0/1 encoding for a bad/good pose image.
    """
    world_label = image.split('_')[0]
    if world_label == 'good':
        return 1
    return 0

In [ ]:
def process_data(form):
    """Function to process the data and save it in a numpy array form (friendlier to neural networks).

    Arguments:
        form {string} -- represents the type of images it receives.
    """
    data = []
    if form == 'good':
        dir = os.listdir(GOOD_POSE_DIR)
    else:
        dir = os.listdir(BAD_POSE_DIR)

    for image in dir:
        label = label_image(image)
        if form == 'good':
            path = os.path.join(GOOD_POSE_DIR, image)
        else:
            path = os.path.join(BAD_POSE_DIR, image)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (IMG_SIZE, IMG_SIZE))
        data.append([np.array(img), label])
    
    if form == 'good':
        np.save('good_pose_data.npy', data)
    else:
        np.save('bad_pose_data.npy', data)

In [7]:
# Loading the data
good_pose = np.load('good_pose_data.npy', allow_pickle=True)
bad_pose = np.load('bad_pose_data.npy', allow_pickle=True)

In [8]:
# Mixing the good pose data and bad pose data
good_and_bad_pose = np.concatenate((good_pose, bad_pose), axis=0)

# Split the data in train/test.
data = good_and_bad_pose[:, 0]
labels = good_and_bad_pose[:, 1]

X_train, X_valid, y_train, y_valid = train_test_split(data, labels, train_size=0.8)

y_train = np.asarray(y_train).astype(np.float32)
y_valid = np.asarray(y_valid).astype(np.float32)

## Running Openpose over the images to get the data

In [9]:
thr = 0.2
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

In [10]:
# Load the weigths
net = cv2.dnn.readNetFromTensorflow('graph_opt.pb')

In [11]:
def get_openpose_data(frame):
    """Function to run the openpose model over an image and get the coordinates for all 19 keypoints.
    
    Arguments:
        frame {list} -- The image that the openpose model is run over.
    
    Returns:
        list -- A list that consists of coordinates of the keypoints.
    """
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv2.dnn.blobFromImage(frame, 1.0, (IMG_SIZE, IMG_SIZE), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]

        points.append([int(x), int(y)] if conf > thr else None)

    return points

In [12]:
def process_openpose_data(data, scaler, train=True):
    """Functions that is processing the data output of openpose.

    Arguments:
        data {list} -- List of keypoints.
        scaler {object} -- Scikit-learn scaler.
        train {boolean} -- Boolean that tells the function to fit the data before transforming.

    Returns:
        {numpy.array} -- Processed data.
    """
    x_data = []

    for idx1, image in enumerate(X_train):
        openpose_data = get_openpose_data(image)
        for idx2, coords in enumerate(openpose_data):
            if coords == None:
                openpose_data[idx2] = [0, 0]
        x_data.append(openpose_data[1:5])
    
    x_data = np.array(x_data).astype(float)
    # Reshape so we can fit the scaler on our data.
    x_data = x_data.reshape(x_data.shape[0], x_data.shape[1] * 2)
    
    # Normalize the data
    if train == True:
        scaler.fit(x_data)
    x_data = scaler.transform(x_data)
    return x_data

In [13]:
scaler = MinMaxScaler()
X_train_openpose = process_openpose_data(X_train, scaler, train=True)
X_valid_openpose = process_openpose_data(X_valid, scaler, train=False)

In [ ]:
# Visualizing the keypoints
for coords in X_train_openpose:
    for coord, body_part in zip(coords, list(BODY_PARTS.keys())[1:5]):
        print(f'{body_part} -> {coord}')
    print('\n')

## The fully conected model that takes the keypoint coordinates

In [14]:
model = Sequential()
model.add(Input(shape=(8)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

In [15]:
model.fit(X_train_openpose, y_train, epochs=200)

Epoch 1/200
4/4 [==============================] - 1s 1ms/step - loss: 0.6758 - accuracy: 0.6757
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 0.6424 - accuracy: 0.5834
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 0.6271 - accuracy: 0.5865
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 0.6209 - accuracy: 0.5702
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 0.5951 - accuracy: 0.6199
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 0.5880 - accuracy: 0.6229
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 0.5744 - accuracy: 0.6715
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.5329 - accuracy: 0.7184
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 0.5249 - accuracy: 0.7527
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.5144 - accuracy: 0.7974
Epoch 11/200
4/4 [===========

In [16]:
output = model.predict(X_valid_openpose)

In [ ]:
print(confusion_matrix(y_valid, output.round()))
print(accuracy_score(y_valid, output.round()))

In [ ]:
# Saving the model
model.save('openpose_bicepscurl_nobg_sigmoid')

## Working on the video preprocessing

In [18]:
def fragment_video(video_name, save_location, interval):
    """Function to fragment video into frames at a given interval of time
        and saves the in a given directory.

    Arguments:
        video_name {string} -- Path to the video.
        save_location {string} -- Path to the save location.
        interval {integer} -- Interval in ms.
    """
    vidcap = cv2.VideoCapture(video_name)
    success, image = vidcap.read()

    # Change the current directory if we are not already there.
    cwd = os.getcwd()
    if cwd != save_location:
        os.chdir(save_location)

    # While we have frames, we read and save them.
    count = 1
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (count*interval))
        cv2.imwrite(f'frame{count}.jpg', image)
        success, image = vidcap.read()
        count += 1

    # Change the directory back.
    os.chdir(cwd)
    

In [19]:
def read_frames(location):
    """Functions that reads the video frames from a director.

    Arguments:
        location {string} -- Path to the directory where the frames are located at.

    Returns:
        {list} -- Resized video frames.
    """
    frames = []
    dir = os.listdir(location)

    for frame in dir:
        path = os.path.join(location, frame)
        frame = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (IMG_SIZE, IMG_SIZE))
        frames.append(frame)
    return frames

In [20]:
fragment_video('D:\College\coding (Python)\PoseNet - Biceps Curl\\videos\\2.mp4', 'frames', 250)
input_data = read_frames('D:\College\coding (Python)\PoseNet - Biceps Curl\\frames')

In [22]:
scaler = MinMaxScaler()
x_valid_video = process_openpose_data(input_data, scaler, train=True)

In [23]:
output = model.predict(x_valid_video)

## Transforming the model into a tensorflow lite model

In [24]:
def convert_to_tflite(model):
    """Functions that converts our keras model to a tensorflow lite model.

    Arguments:
        model {object} -- Keras model that needs to be converted.
    """
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)

In [25]:
convert_to_tflite(model, 'D:\College\coding (Python)\PoseNet - Biceps Curl\\tflite')

INFO:tensorflow:Assets written to: C:\Users\Dragos\AppData\Local\Temp\tmpskx1x6v9\assets
